In [2]:
# selenium 불러오기 
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from datetime import datetime, timedelta 
from datetime import date as dt
import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [2]:
#driver = webdriver.Chrome(executable_path=r"C:\webdrivers\chromedriver.exe")

In [15]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('window-size=1920x1080')
driver = webdriver.Chrome(executable_path=r'C:\webdrivers\chromedriver.exe', chrome_options=chrome_options)
# TBC 접속하기 
driver.implicitly_wait(3)
driver.get('https://www.tbc.net.tw/Epg/')
wait = WebDriverWait(driver, 500000)

<ipython-input-15-09481a101d7e>:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=r'C:\webdrivers\chromedriver.exe', chrome_options=chrome_options)


In [60]:
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight/12);")
#driver.execute_script("window.scrollTo(0, window.scrollY + 200);")

In [4]:
def update_date(driver,start_date): #ex)2021/02/28
  
  #change hour to initial 00:00
  hour_pick = Select(driver.find_element_by_id("SearchParameter_Hour"))
  hour_pick.select_by_visible_text("00:00") 
  #change date
  date_pick = driver.find_element_by_id('SearchParameter_StartDate')
  upinpt = driver.execute_script("arguments[0].setAttribute('value','{a_date}')".format(a_date = start_date), date_pick)
  #driver.execute_script("($'#SearchParameter_Hour').val('0');")
  #driver.execute_script("($'#SearchParameter_StartDate').val({a_date});".format(a_date = date))
  update_click = driver.find_element_by_xpath('//*[@id="epgForm"]/div[1]/div[4]/a[2]')
  # result = driver.execute_script("arguments[0].click();", update_click)
  print('Date is',start_date)
  print('Hour is 00:00',)
  print("Succesfuly updated")
  upclic = driver.execute_script("arguments[0].click();", update_click)
  return (start_date, upinpt,"00:00",upclic)

In [5]:
def next_tmLine(start_hr, start_date,end_date):
  
  opt_seletd = Select(driver.find_element_by_id("SearchParameter_Hour"))
  seletd_tm = opt_seletd.first_selected_option
  print("current time starting "
        "point :{}".format(seletd_tm.text))
  prsent_tm = seletd_tm.text
  if (prsent_tm != "18:00"):
    print("you are eligible to increment hours")
    #to convert string to time format HH:MM
   # start_tm = #"00:00"
    weighted_tm = 6
    int_tm = int(start_hr.rsplit(':', 1)[0])
    added_tm = int_tm + weighted_tm
    start_hr = "{}:00".format(str(added_tm).zfill(2))
    print("Date is",start_date)
    print("new start hour is :",start_hr)
   
    opt_seletd.select_by_visible_text(start_hr) 
    update_click = driver.find_element_by_xpath('//*[@id="epgForm"]/div[1]/div[4]/a[2]')
    uphr = driver.execute_script("arguments[0].click();", update_click)
    print("Succesfuly updated")
      
    return (prsent_tm, start_hr,str(start_date), uphr, uphr)
  else:
    print("pls, use update_date_function instead")
    #start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
    start_date = start_date.strftime("%Y/%m/%d")
    #start_date += datetime.timedelta(days=1)
    start_date = str(start_date)
    print("Date is",start_date)
    out = update_date(driver,start_date)
    print("out[0]:",out[0])
    return (out[0],"00:00", out[0])


In [5]:
# hr = '00:00'
# init_day = '2021/03/01'
# end_day = '2021/03/10'
# update_date(driver, init_day)
# #next_tmLine(hp,tp,fp)
# while init_day < end_day:
#  get = next_tmLine(hr,init_day,end_day)
#  init_day = get[1]
#  end_day = get[2]

In [7]:
#@markdown check each day info
def today_info():
  """Wait and Check the len of EPG and the group of corresponding CNT """
  try:  
    epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
    cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
    if (len(epg) != len(cnt)):  
  # we can pass the message in the raise statement  
      raise ValueError("the number of channels and contents lists don't match!!") 
    else:
      #print('the num of channels and the num of lists of contents on the table match! Keep crawling!') 
      #print("the each num of channel and lists of contents is {}".format(len(epg)))
      print("channel and contents nm {} matches".format(len(epg)))
  except ValueError as e:  
      print(e) 
  return (None)

In [76]:
a_l = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/ul[3]/li[2]/a')))
a_ctn = webdriver.ActionChains(driver).move_to_element(a_l).click()
a_ctn.perform()

element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="epg_popup"]/div/div[1]/div[3]/a')))
element.click()

#@title Create CLOSE btn!!
close_btn = '//*[@id="epg_popup"]/div/div[1]/div[3]/a'
close_btn_ele = driver.find_element_by_xpath(close_btn)
close_pop = webdriver.ActionChains(driver).move_to_element(close_btn_ele).click()
close_pop.perform()



In [71]:
cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
i=1
a_ctn_list = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)))) 
j=1
a_ctn = webdriver.ActionChains(driver).move_to_element(a_ctn_list[j]).click()

In [73]:
a_ctn_list[j]                                                                                                                                    w  

<selenium.webdriver.remote.webelement.WebElement (session="2a1f1352b73e081cf5ce1330588e7cfd", element="f83e0f3d-23f3-4202-932c-929984c49ecb")>

In [28]:
def get_Epg_info():

    epg_info = []
    
    epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
    cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
    for i in range(2,259): #range(1,259) #range(len(epg))
      
      # if (i=18):
      #   driver.execute_script("window.scrollTo(0, document.body.scrollHeight/12);")
                
      tv_name = epg[i].text.split('\n') #get the channel name //*[@id="content"]/ul[3]/li[3]/a
      
      #a_ctn_list = wait.until(
      #  EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)))) # match to the corresponding CH
      a_ctn_list = cnt[i].find_elements_by_xpath('//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)) # match to the corresponding CH#

      ctn_info = []
      for j in range(len(a_ctn_list)): #get the each info from the correspoing lists of contents
        a_ctn = webdriver.ActionChains(driver).move_to_element(a_ctn_list[j]).click() #--》이것을 아래의 element_to_be_clickable로 바뀌기
        a_ctn.perform()
        #a_ctn = wait.until(
        #  EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/ul[{:d}]/li[{:d}]/a'.format(i+1,j+1))))
        #a_ctn.click()
        try: # if one of info is empty, we just leave it as a blank
          #driver.implicitly_wait(3)
          # ctn_info.append(tv_name +
          #             [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_name"]'))).text] +
          #             wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_moreinfo"]/p'))).text.split('\n') +
          #             [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_desc"]'))).text.replace("\n", " ")] +
          #             [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_date"]'))).text.replace("\n", " ")] +  #--> 다 wait으로 바꾸기
          #             wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_time"]'))).text.split()
          #             )
          ctn_info.append(tv_name +
            [driver.find_element_by_xpath('//*[@id="prg_name').text] +
            driver.find_element_by_xpath('//*[@id="prg_moreinfo"]/p').text.split('\n') +
            [driver.find_element_by_xpath('//*[@id="prg_desc"]').text.replace("\n", " ")] +
            [driver.find_element_by_xpath('//*[@id="prg_date"]').text.replace("\n", " ")] +  #--> 다 wait으로 바꾸기
            driver.find_element_by_xpath('//*[@id="prg_time"]').text.split()
            )
        
        except (NoSuchElementException) as e :
          # Just append a None or ""
          ctn_info.append(['']+['']+['']+['']+['']+[''])
        
        # epg_info.append({'channel': tv_name,
        #                   'prog': list(zip(*ctn_info))[0],
        #                   'genre': list(zip(*ctn_info))[1],
        #                   'desc': list(zip(*ctn_info))[2],
        #                   'time': list(zip(*ctn_info))[4]
        #                 })    
        epg_info.extend(ctn_info)                                    #//*[@id="epg_popup"]/div/div[1]/div[3]/a
        element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="epg_popup"]/div/div[1]/div[3]/a')))
        element.click()
        #driver.implicitly_wait(3) 

      # if i % 2 == 0:
      #   driver.execute_script("window.scrollTo(0, window.scrollY + 200);")

    return (epg_info)

# def get_Epg_info():

#     epg_info = []
    
#     epg = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="epg"]/div[4]/div/ul/li')))
#     cnt = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul')))
#     for i in range(50): #range(len(epg))
      
#       tv_name = epg[i].text.split('\n') #get the channel name
#       a_ctn_list = WebDriverWait(cnt[i], 50).until(
#           EC.presence_of_all_elements_located((By.XPATH, '//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)))) # match to the corresponding CH
#       # equals --> a_ctn_list = cnt[i].find_elements_by_xpath('//*[@id="content"]/ul[{:d}]/li/a'.format(i+1)) # match to the corresponding CH#
#       ctn_info = []
#       for j in range(len(a_ctn_list)): #get the each info from the correspoing lists of contents
#         a_ctn = webdriver.ActionChains(driver).move_to_element(a_ctn_list[j]).click() #--》이것을 아래의 element_to_be_clickable로 바뀌기
#         a_ctn.perform()
#         try: # if one of info is empty, we just leave it as a blank
#           driver.implicitly_wait(3)
#           ctn_info.append([wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_name"]'))).text] +
#                       wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_moreinfo"]/p'))).text.split('\n') +
#                       [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_desc"]'))).text.replace("\n", " ")] +
#                       [wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_date"]'))).text.replace("\n", " ")] +  #--> 다 wait으로 바꾸기
#                       wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="prg_time"]'))).text.split()
#                       )
        
#         except (NoSuchElementException, TimeoutException) as e :
#           # Just append a None or ""
#           ctn_info.append(['']+['']+['']+['']+[''])
        
#         epg_info.append({'channel': tv_name,
#                           'prog': list(zip(*ctn_info))[0],
#                           'genre': list(zip(*ctn_info))[1],
#                           'desc': list(zip(*ctn_info))[2],
#                           'time': list(zip(*ctn_info))[4]
#                         })                                         #//*[@id="epg_popup"]/div/div[1]/div[3]/a
#         element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="epg_popup"]/div/div[1]/div[3]/a')))
#         element.click()
#         driver.implicitly_wait(3) 
#     return (epg_info)

In [23]:
test= get_Epg_info()

In [30]:
hr = '00:00'
init_day = '2021/03/01'
end_day = '2021/03/08'
update_date(driver, init_day)
#next_tmLine(hp,tp,fp)
EPG_per = []
while init_day < end_day:
 EPG_per.extend(get_Epg_info())
 get = next_tmLine(hr,init_day,end_day)
 
 hr = get[1]
 init_day = get[2]
 

Date is 2021/03/01
Hour is 00:00
Succesfuly updated


In [18]:
EPG_per

[[['3', '南桃園公用頻道'],
  '公益時段/生活資訊',
  '類別：綜藝/娛樂',
  '公益時段/生活資訊',
  '2021/03/01',
  '00:00~05:00'],
 [['3', '南桃園公用頻道'],
  '公益時段/生活資訊',
  '類別：綜藝/娛樂',
  '公益時段/生活資訊',
  '2021/03/01',
  '00:00~05:00'],
 [['3', '南桃園公用頻道'], '公益廣告', '類別：綜藝/娛樂', '公益廣告', '2021/03/01', '05:00~06:00'],
 [['5', 'CNN'],
  'Reliable Sources',
  '類別：新聞',
  'Reliable Sources',
  '2021/03/01',
  '00:00~00:30'],
 [['5', 'CNN'],
  'Reliable Sources',
  '類別：新聞',
  'Reliable Sources',
  '2021/03/01',
  '00:00~00:30'],
 [['5', 'CNN'],
  'Reliable Sources',
  '類別：新聞',
  'Reliable Sources',
  '2021/03/01',
  '00:30~01:00'],
 [['5', 'CNN'],
  'Reliable Sources',
  '類別：新聞',
  'Reliable Sources',
  '2021/03/01',
  '00:00~00:30'],
 [['5', 'CNN'],
  'Reliable Sources',
  '類別：新聞',
  'Reliable Sources',
  '2021/03/01',
  '00:30~01:00'],
 [['5', 'CNN'],
  'World Sport',
  '類別：新聞',
  'World Sport',
  '2021/03/01',
  '01:00~01:30'],
 [['5', 'CNN'],
  'Reliable Sources',
  '類別：新聞',
  'Reliable Sources',
  '2021/03/01',
  '00:00~00:30'],
 

In [19]:

#@title Save the Genres variables
#save variables
import pickle
with open('EPG_per_test_prepro','wb') as f:
  pickle.dump(EPG_per, f)

In [27]:
import pandas as pd
df = pd.DataFrame(test)
#df.columns =['prog_name','genre','desc']
print(len(df.index))

318


In [28]:
df

,0,1,2,3,4,5,6
0,3,南桃園公用頻道,公益時段/生活資訊,類別：綜藝/娛樂,公益時段/生活資訊,2021/03/02,00:00~05:00
1,3,南桃園公用頻道,公益時段/生活資訊,類別：綜藝/娛樂,公益時段/生活資訊,2021/03/02,00:00~05:00
2,3,南桃園公用頻道,公益廣告,類別：綜藝/娛樂,公益廣告,2021/03/02,05:00~06:00
3,5,CNN,Connect the World,類別：新聞,Connect the World,2021/03/02,00:00~00:30
4,5,CNN,Connect the World,類別：新聞,Connect the World,2021/03/02,00:00~00:30
...,...,...,...,...,...,...,...
313,12,華視,女王當家:116,類別：知性（知識）,節目邀請各領域達人、專業主廚，與觀眾朋友分享如何全方位照料家人健康與聰明消費。從輕熟女好媳婦...,2021/03/02,02:00~03:00
314,12,華視,華視夜間新聞,類別：新聞,提供觀眾朋友重要的國內外大事。,2021/03/02,03:00~04:00
315,12,華視,女王當家:79,類別：知性（知識）,節目邀請各領域達人、專業主廚，與觀眾朋友分享如何全方位照料家人健康與聰明消費。從輕熟女好媳婦...,2021/03/02,04:00~05:00
316,12,華視,英語口語訓練（一）（商專）:1,類別：教育學習、知性（知識）,英語口語訓練（一）（商專）,2021/03/02,05:00~05:30
